### <center> Point Cloud Registration: Coherent Point Drift </center>

<div style="text-align: justify">Point set registration is a key component in many computer vision tasks. The goal of point set registration is to assign correspondences between two sets of points and to recover the transformation that maps one point set to the other. Multiple factors, including an unknown non-rigid spatial transformation, large dimensionality of point set, noise and outliers, make the point set registration a challenging problem. We introduce a probabilistic method, called the Coherent Point Drift (CPD) algorithm, for both rigid and non-rigid point set registration. We consider the alignment of two point sets as a probability density estimation problem. We fit the GMM centroids (representing the first point set) to the data (the second point set) by maximizing the likelihood. We force the GMM centroids to move coherently as a group to preserve the topological structure of the point sets. In the rigid case, we impose the coherence constraint by re-parametrization of GMM centroid locations with rigid parameters and derive a closed form solution of the maximization step of the EMalgorithm in arbitrary dimensions. In the non-rigid case, we impose the coherence constraint by regularizing the displacement field and using the variational calculus to derive the optimal transformation. We also introduce a fast algorithm that reduces the method computation complexity to linear. We test the CPD algorithm for both rigid and non-rigid transformations in the presence of noise, outliers and missing points, where CPD shows accurate results and outperforms current state-of-the-art methods.</div>

<img src="https://prs.igp.ethz.ch/research/completed_projects/automatic_registration_of_point_clouds/_jcr_content/par/fullwidthimage/image.imageformat.lightbox.1637563831.png">

### 1. Importing Libs

In [2]:
# Matrix and quaternions manipulations
import numpy as np
from pyquaternion import Quaternion

In [3]:
# Registration
from functools import partial
from pycpd import rigid_registration
import time

In [5]:
# Visualization
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import make_subplots
init_notebook_mode(connected=True)

### 2. Functions

In [6]:
def rotateQuaternion(x,y,z,q):
    xyz = np.transpose(np.vstack([x,y,z]))
    listxyz = []
    for i in range(xyz.shape[0]):
        v = xyz[i,:]
        v_prime = q.rotate(v)
        listxyz.append(v_prime)
    xyz_prime = np.vstack(listxyz)
    return xyz_prime[:,0], xyz_prime[:,1], xyz_prime[:,2]

In [7]:
def trace3D(PointCloud, color, size, name):
    trace = go.Scatter3d(
        x=PointCloud[:,0],
        y=PointCloud[:,1],
        z=PointCloud[:,2],
        name=name,
        mode='markers',
        marker=dict(
            size=size,
            line=dict(
                color=color,
                width=0.5
            ),
            opacity=0.8
        )
    )
    return trace

In [8]:
def runregistration(iteration, error, X, Y, val):
    print('iteration: ' + str(iteration) + ', error: ' + str(error))

### 3. Generate Source and Target Point Clouds

In [9]:
# Generate XY Grid
initX = 0.0
totalX = 20
endX = 10

initY = 0.0
totalY = 10
endY = 10

listX = []
for i in range(totalY):
    listX = listX + np.linspace(initX, endX, totalX).tolist()
x = np.array(listX)

listY = []
listValY = np.linspace(initY, endY, totalY)
for val in listValY:
    listY = listY + [val for i in range(totalX)]
y = np.array(listY)

print(x.shape)
print(y.shape)

(200,)
(200,)


In [10]:
# Getting Z from plane Equation
# Plane ax + by + cz = d ==> z = (1/c)(d - ax -by)
a = 0.0
b = 1.0
c = 1.0
d = 1.0

z = (1.0/c)*(d - a*x - b*y)
z_noise = z + np.random.rand(len(z))

In [11]:
numperpi = 3.141592654
np.set_printoptions(suppress=True) # Suppress insignificant values for clarity
q = Quaternion(axis=[1, 0, 0], angle=numperpi/4.0) # Rotate 0 about x=y=z
x_prime, y_prime, z_prime = rotateQuaternion(x,y,z_noise,q)

In [12]:
X = np.transpose(np.vstack([x, y, z]))                   # Target
Y = np.transpose(np.vstack([x_prime, y_prime, z_prime])) # Source

In [13]:
fig = make_subplots(rows=1, cols=1, specs=[[{'is_3d': True}]])

traceSource = trace3D(X, color='red', size=6, name='Source')
traceTarget = trace3D(Y, color='blue', size=6, name='Target')

fig.append_trace(traceSource, 1, 1)
fig.append_trace(traceTarget, 1, 1)

iplot(fig, filename='simple-3d-scatter')

This is the format of your plot grid:
[ (1,1) scene1 ]



### 4. Registration

In [14]:
max_iterations=200
tolerance = 0.001
reg = rigid_registration(**{ 'X': X, 'Y': Y, 'max_iterations': max_iterations, 'tolerance':tolerance})
callback = partial(runregistration, val=1)
reg.register(callback)

iteration: 1, error: 2102.5258242342147
iteration: 2, error: 213.90840741014676
iteration: 3, error: 92.8510782635426
iteration: 4, error: 116.70218394437404
iteration: 5, error: 130.52537930525216
iteration: 6, error: 134.78434972017305
iteration: 7, error: 133.85618946444453
iteration: 8, error: 130.61915848690845
iteration: 9, error: 128.67386978636804
iteration: 10, error: 138.917007044555
iteration: 11, error: 170.72126215473727
iteration: 12, error: 188.06636957812384
iteration: 13, error: 144.79131435975347
iteration: 14, error: 78.13784286384214
iteration: 15, error: 32.3611576135944
iteration: 16, error: 10.231052591268053
iteration: 17, error: 3.0132388707930886
iteration: 18, error: 0.9389705341266108
iteration: 19, error: 0.3149986484064584
iteration: 20, error: 0.11314616421236678
iteration: 21, error: 0.043115598623444384
iteration: 22, error: 0.017239216567077165
iteration: 23, error: 0.007158931564731574
iteration: 24, error: 0.0030633917000386646
iteration: 25, error: 

(array([[-0.01098487,  0.00557002,  0.61261634],
        [ 0.5067351 ,  0.00631597,  1.46296538],
        [ 1.03089234,  0.0074978 ,  1.48116441],
        [ 1.55810233,  0.00888634,  1.10473137],
        [ 2.08048005,  0.00994768,  1.35297003],
        [ 2.60741865,  0.01131784,  1.0116202 ],
        [ 3.1345739 ,  0.01270268,  0.64226257],
        [ 3.65354406,  0.01353328,  1.33099992],
        [ 4.18350652,  0.0151082 ,  0.59875288],
        [ 4.70182761,  0.01589485,  1.37139438],
        [ 5.23150344,  0.01745036,  0.6762002 ],
        [ 5.75136901,  0.01834159,  1.24918693],
        [ 6.27595194,  0.01955225,  1.21235527],
        [ 6.79778849,  0.02057694,  1.53055122],
        [ 7.32735631,  0.02212514,  0.84932036],
        [ 7.84954838,  0.0231739 ,  1.12155822],
        [ 8.37170666,  0.02422038,  1.39816299],
        [ 8.90216925,  0.02582917,  0.60126442],
        [ 9.42415774,  0.02686415,  0.89981898],
        [ 9.94714972,  0.02796708,  1.06865123],
        [-0.00907052

### 5. Get Transformation Scales, Rotation and Translation

In [15]:
s, R, t = reg.get_registration_parameters()
print('Scale: '+str(s))
print('Rotation: ' +str(R))
print('Translation: '+str(t))

Scale: 0.9961389921773769
Rotation: [[ 0.9999675   0.00226831  0.00773664]
 [ 0.00386304  0.70747524 -0.70672757]
 [-0.00707656  0.70673449  0.70744348]]
Translation: [-0.00230655  0.00615766 -0.50923742]


### 6. Transform Source according to final Transformation

In [16]:
YTransformed = s * np.dot(Y, R) + t

### 7. Visualize Source, Target and Final Transformation

In [17]:
fig = make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])

traceSource = trace3D(X, color='red', size=6, name='Source')
traceTarget = trace3D(Y, color='blue', size=6, name='Target')
traceFinal = trace3D(YTransformed, color='red', size=6, name='Final')

fig.append_trace(traceSource, 1, 1)
fig.append_trace(traceTarget, 1, 1)
fig.append_trace(traceSource, 1, 2)
fig.append_trace(traceFinal, 1, 2)

iplot(fig, filename='simple-3d-scatter')

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]

